In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A

import os
from glob import glob
import requests
import random
from time import time
from enum import Enum
from pprint import pprint

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, bias=True)
        self.bn1 = nn.BatchNorm2d(num_features=3)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=3, bias=False)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        return F.relu(self.conv2(x))

```
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
            self.layer = 

    def forward(self, x):
        x = self.layer(x)
        return x
```

In [3]:
model = Model()
model

Model(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), bias=False)
)

### named_parameters()

In [8]:
for param, weight in model.named_parameters():
    print(param)
    print(weight.size())
    print(weight)
    print('#'*100)

conv1.weight
torch.Size([3, 1, 3, 3])
Parameter containing:
tensor([[[[ 0.1688, -0.0547, -0.1594],
          [-0.0223,  0.2883,  0.0153],
          [-0.0523,  0.1475, -0.0500]]],


        [[[ 0.0070, -0.1964,  0.0903],
          [ 0.1511, -0.1235,  0.2422],
          [-0.0602,  0.0905, -0.0266]]],


        [[[ 0.2699,  0.0363,  0.2206],
          [-0.1975,  0.1651, -0.1708],
          [-0.1452, -0.1330, -0.2261]]]], requires_grad=True)
####################################################################################################
conv1.bias
torch.Size([3])
Parameter containing:
tensor([ 0.2864, -0.0154, -0.2926], requires_grad=True)
####################################################################################################
bn1.weight
torch.Size([3])
Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
####################################################################################################
bn1.bias
torch.Size([3])
Parameter containing:
tensor([0., 0

In [7]:
print(model.conv1.weight)

Parameter containing:
tensor([[[[ 0.1688, -0.0547, -0.1594],
          [-0.0223,  0.2883,  0.0153],
          [-0.0523,  0.1475, -0.0500]]],


        [[[ 0.0070, -0.1964,  0.0903],
          [ 0.1511, -0.1235,  0.2422],
          [-0.0602,  0.0905, -0.0266]]],


        [[[ 0.2699,  0.0363,  0.2206],
          [-0.1975,  0.1651, -0.1708],
          [-0.1452, -0.1330, -0.2261]]]], requires_grad=True)


# Save model

In [9]:
save_folder = './runs'
save_path = os.path.join(save_folder, 'best.pth')
os.makedirs(save_folder, exist_ok=True)

torch.save(model.state_dict(), save_path)

```
torch.save(model.state_dict(), save_path)
```

# Load model

In [11]:
new_model = Model()
new_model.load_state_dict(torch.load(save_path))

<All keys matched successfully>

```
new_model.load_state_dict(torch.load(save_path))
```

In [14]:
for (name, trained_weight), (_, saved_weight) in zip(model.named_parameters(), new_model.named_parameters()):
    is_equal = torch.equal(trained_weight, saved_weight)
    print(f'{name} from model and new_model are matched - {is_equal}')

conv1.weight from model and new_model are matched - True
conv1.bias from model and new_model are matched - True
bn1.weight from model and new_model are matched - True
bn1.bias from model and new_model are matched - True
conv2.weight from model and new_model are matched - True


### state_dict()

In [15]:
for param, weight in model.state_dict().items():
    print(param)
    print(weight.size())
    print(weight)
    print('#'*100)

conv1.weight
torch.Size([3, 1, 3, 3])
tensor([[[[ 0.1688, -0.0547, -0.1594],
          [-0.0223,  0.2883,  0.0153],
          [-0.0523,  0.1475, -0.0500]]],


        [[[ 0.0070, -0.1964,  0.0903],
          [ 0.1511, -0.1235,  0.2422],
          [-0.0602,  0.0905, -0.0266]]],


        [[[ 0.2699,  0.0363,  0.2206],
          [-0.1975,  0.1651, -0.1708],
          [-0.1452, -0.1330, -0.2261]]]])
####################################################################################################
conv1.bias
torch.Size([3])
tensor([ 0.2864, -0.0154, -0.2926])
####################################################################################################
bn1.weight
torch.Size([3])
tensor([1., 1., 1.])
####################################################################################################
bn1.bias
torch.Size([3])
tensor([0., 0., 0.])
####################################################################################################
bn1.running_mean
torch.Size([3])
tensor

`named_parameters()`: returns only parameters

`state_dict()`: returns both parameters and buffers

In [18]:
print([name for (name, param) in model.named_parameters()])
print(list(model.state_dict().keys()))

['conv1.weight', 'conv1.bias', 'bn1.weight', 'bn1.bias', 'conv2.weight']
['conv1.weight', 'conv1.bias', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'conv2.weight']


# cuda

In [24]:
data = torch.randn(2, 2, device=torch.device('cuda'))
print(data.device)

cuda:0


`data = torch.randn(2,2).cuda()`는 CPU 메모리에 텐서를 만들고나서 GPU로 옯기는 것. 

Cost inefficient.

In [20]:
# 모델의 모든 파라미터와 버퍼를 CPU 메모리로 옮기기
model.cpu()

for param in model.parameters():
    print(param.device)

cpu
cpu
cpu
cpu
cpu


In [21]:
model.cuda()

for param in model.parameters():
    print(param.device)

cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


In [22]:
device_options = ['cpu', 'cuda']
device = torch.device(device_options[1])

model.to(device)

for param in model.parameters():
    print(param.device)

cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


# Forward

`nn.Module`을 상속한 객체를 직접 호출할 때 수행하는 연산

In [27]:
input = torch.randn(1, 1, 12, 12, device=device)
model.to(device)
output = model(input)
print(output.size())

torch.Size([1, 5, 8, 8])
